If you have a Cayley database, you can build a machine learning model for such a task:

Given a partially filled Cayley table of a semigroup, restore the full one.

It should be mentioned that a partially filled table sometimes can be filled in several ways to a full associative table. We will consider all such solutions as equally valid.

In `neural-semigroups` package we use `torch` for building deep learning models.

First of all, we need to get some training and validation data.
In this example, we take semigroups of 7 items, and hold 100 Cayley tables (each representing a different class of equivalent semigrous) as our training data, and another 100 tables as validation.
This is a really small fraction of all tables of 7 elements available (there are 836021 of them up to equivalence).

Here we construct `DataLoaders` for `torch` which will feed a training pipeline with 48 tables at a time.
This number (batch size) can be changed for fine-tuning the model's quality.

In [1]:
from neural_semigroups.training_helpers import get_loaders

cardinality = 7
data_loaders = get_loaders(
    cardinality=cardinality,
    batch_size=48,
    train_size=100,
    validation_size=100
)


generating test cubes: 100%|██████████| 835821/835821 [00:26<00:00, 32105.40it/s]


We model each input Cayley table as a three index tensor $a_{ijk}$ such that

$a_{ijk}=P\left\{e_ie_j=e_k\right\}$

where $e_i$ are elements of a semigroup.

In our training data all $a_{ijk}$ are either zeros or ones, so probability distributions involved are degenerate.

When we need to hide a cell with indices $i,j$ from an original Cayley table we set

$a_{ijk}=\dfrac1n$

where $n$ is the semigroup's cardinality. Thus we set a probability distribution of the multiplication result $e_ie_j$ to discrete uniform.

We choose a simple denoising autoencoder as an architecture for our neural network. It simply gets an input tensor of zeros and ones, hide 50% of input cells in a manner described earlier, and applies a linear transformation into a lower dimension ($n^2$ but these also can be tuned) with a simple `RuLU` non-linearity. Then another linear transformation with `ReLU` is applied to return back to the original $n^3$ dimension. We also apply batch norm here. See the package code for the details.

In [2]:
from neural_semigroups import MagmaDAE

dae = MagmaDAE(
    cardinality=cardinality,
    hidden_dims=[
        cardinality ** 2
    ],
    corruption_rate=0.5
)

In [3]:
dae

MagmaDAE(
  (decoder_layers): Sequential(
    (linear10): Linear(in_features=49, out_features=343, bias=True)
    (relu10): ReLU()
    (bn10): BatchNorm1d(343, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (encoder_layers): Sequential(
    (linear00): Linear(in_features=343, out_features=49, bias=True)
    (relu00): ReLU()
    (bn00): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

During the training process we try to minimize the Kullback-Leibler distance between two probabilistic cubes (the input and output ones of the DAE).

In [4]:
import torch
from torch import Tensor
from torch.nn.functional import kl_div

def loss(prediction: Tensor, target: Tensor) -> Tensor:
    return kl_div(torch.log(prediction), target, reduction="batchmean")


We use `pytorch-ignite` to write less boilerplate code for a training pipeline.

In [5]:
from ignite.engine import create_supervised_evaluator
from ignite.metrics.loss import Loss

evaluator = create_supervised_evaluator(
    dae,
    metrics={"loss": Loss(loss)}
)

Now it's time to run a pipeline! Here you can tune the learning schedule for better results.

You can construct your own pipeline if you don't want to import one provided by the package.

In the next three cells we will run `tensorboard` to show training/validation curves during training process.

In [6]:
%load_ext tensorboard

In [7]:
!rm -rf runs

In [9]:
%tensorboard --logdir runs --host 0.0.0.0

In [10]:
%%time
from neural_semigroups.training_helpers import learning_pipeline

params = {
    "learning_rate": 0.005,
    "epochs": 1000,
    "cardinality": cardinality
}
learning_pipeline(params, dae, evaluator, loss, data_loaders)

CPU times: user 58.4 s, sys: 1.92 s, total: 1min
Wall time: 40.2 s


And here is the report of results. It seems to be quite impressive. For it we got 1000 of random Cayley tables (for different equivalent classes as always) and constructed 'puzzles' from it.

Level of difficulty for a puzzle is a number of hidden cells. A puzzle is considered to be solved if the model returns a full associative table.

We see that the model generalizes well (it was trained only on 100 tables).

In [11]:
from neural_semigroups.utils import print_report
from neural_semigroups import CayleyDatabase

cardinality = 7
cayley_db = CayleyDatabase(cardinality)
cayley_db.load_model(f"semigroups.{cardinality}.model")
print_report(cayley_db.testing_report)

generating and solving puzzles: 100%|██████████| 1000/1000 [00:57<00:00, 17.49it/s]


,puzzles,solved,(%),hidden cells,guessed,in %
level,,,,,,
1,1000,892,89,1000,892,89
2,1000,840,84,2000,1804,90
3,1000,810,81,3000,2704,90
4,1000,778,77,4000,3597,89
5,1000,759,75,5000,4507,90
6,1000,750,75,6000,5409,90
7,1000,740,74,7000,6277,89
8,1000,747,74,8000,7189,89
9,1000,736,73,9000,8033,89
